# Discrete Optimization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gaussian_process.GPfunctions as gp
from gaussian_process import GaussianProcess
from gaussian_process.kernels import SquaredExponentialKernel
from acquisition import GP_UCB, GP_LCB

In [ ]:
# Objective function

objectiveFunction = lambda x: -x * np.sin(x)
objectiveFunctionDerivative = lambda x: -x * np.cos(x) - np.sin(x)

X = np.linspace(start=-3.0, stop=3.0, num=1_000)
y = np.squeeze(objectiveFunction(X))

## Maximization

In [ ]:
X_sample = np.array([0.0, 1.0])
y_sample = objectiveFunction(X_sample)

x_best = X_sample[np.argmax(y_sample)]
y_best = np.max(y_sample)

for t in range(30):
    kernel = SquaredExponentialKernel(l=np.abs(X[0] - X[-1]) / (len(X_sample - 1)))
    GP_posterior = GaussianProcess(kernel=kernel, x_known=X_sample, f_known=y_sample)
    mean, variance = GP_posterior(X)

    fig, (ax1) = plt.subplots(1, 1, sharey=True)
    gp.plot_objective(ax1, X, y, X_sample, y_sample)
    gp.plot_gp(ax1, X, mean, variance)

    acquisitionFunction = GP_UCB(GP_posterior, t, X[0].ndim)
    acquisitionValues = np.array([acquisitionFunction(x) for x in X])

    maxAquisition = np.nanargmax(acquisitionValues)

    ax1.plot(X, acquisitionValues, label="Acquisition")

    gp.plot_label(ax1, f"{len(X_sample)} data points")
    plt.show()

    x_new = X[maxAquisition]
    y_new = objectiveFunction(x_new)
    if x_new in X_sample:
        break
    X_sample = np.append(X_sample, x_new)
    y_sample = np.append(y_sample, y_new)
    if y_new > y_best:
        x_best, y_best = x_new, y_new
    print(f"best: x={x_best}, y={y_best}")

print(f"best: ({x_best},{y_best})")

## Minimization

In [ ]:
X_sample = np.array([0.0, 1.0])
y_sample = objectiveFunction(X_sample)

x_best = X_sample[np.argmin(y_sample)]
y_best = np.min(y_sample)

for t in range(30):
    kernel = SquaredExponentialKernel(l=np.abs(X[0] - X[-1]) / (len(X_sample) - 1))
    GP_posterior = GaussianProcess(kernel=kernel, x_known=X_sample, f_known=y_sample)
    mean, variance = GP_posterior(X)

    fig, (ax1) = plt.subplots(1, 1, sharey=True)
    gp.plot_objective(ax1, X, y, X_sample, y_sample)
    gp.plot_gp(ax1, X, mean, variance)

    acquisitionFunction = GP_LCB(GP_posterior, t, X[0].ndim)
    acquisitionValues = np.array([acquisitionFunction(x) for x in X])

    maxAquisition = np.nanargmax(acquisitionValues)

    ax1.plot(X, acquisitionValues, label="Acquisition")

    gp.plot_label(ax1, f"{len(X_sample)} data points")
    plt.show()

    x_new = X[maxAquisition]
    y_new = objectiveFunction(x_new)
    if x_new in X_sample:
        print("Terminating due to insufficient acquisition function")
        break
    X_sample = np.append(X_sample, x_new)
    y_sample = np.append(y_sample, y_new)
    if y_new < y_best:
        x_best, y_best = x_new, y_new
    print(f"best: x={x_best}, y={y_best}")

print(f"best: ({x_best},{y_best})")

## Minimization with gradient

In [ ]:
X_sample = np.array([0.0, 1.0])
y_sample = objectiveFunction(X_sample)
g_sample = objectiveFunctionDerivative(X_sample)

x_best = X_sample[np.argmin(y_sample)]
y_best = np.min(y_sample)

for t in range(100):
    kernel = SquaredExponentialKernel(
        np.abs(X[0] - X[-1]) / (2.0 * (len(X_sample) - 1))
    )
    GP_posterior = GaussianProcess(
        kernel=kernel, x_known=X_sample, f_known=y_sample, g_known=g_sample
    )
    mean, variance = GP_posterior(X)

    fig, (ax1) = plt.subplots(1, 1, sharey=True)
    gp.plot_objective(ax1, X, y, X_sample, y_sample)
    gp.plot_gp(ax1, X, mean, variance)

    acquisitionFunction = GP_LCB(GP_posterior, t, X[0].ndim)
    acquisitionValues = np.array([acquisitionFunction(x) for x in X])

    maxAquisition = np.nanargmax(acquisitionValues)

    ax1.plot(X, acquisitionValues, label="Acquisition")

    gp.plot_label(ax1, f"{len(X_sample)} data points")
    plt.show()

    x_new = X[maxAquisition]
    y_new = objectiveFunction(x_new)
    g_new = objectiveFunctionDerivative(x_new)
    if x_new in X_sample:
        print("Terminating due to insufficient acquisition function")
        break
    X_sample = np.append(X_sample, x_new)
    y_sample = np.append(y_sample, y_new)
    g_sample = np.append(g_sample, g_new)

    if y_new < y_best:
        x_best, y_best = x_new, y_new
    print(f"best: x={x_best}, y={y_best}")

print(f"best: ({x_best},{y_best})")